# Generate FIR Filter Coefficients

Demonstrates how to generate precomputed/predefined FIR filter coefficients for various resampling scenarios based on input and output sample rates. The coefficients are formatted in a convenient way to facilitate adding them to a C/C++ file as compile-time constants. 

In [57]:
from fractions import Fraction
import numpy as np
import os
from scipy import signal

In [58]:
INPUT_SAMPLE_RATES_STR = os.environ.get('INPUT_SAMPLE_RATE', '1140000,1140000,1000000,228000,200000')
OUTPUT_SAMPLE_RATES_STR = os.environ.get('OUTPUT_SAMPLE_RATE', '228000,200000,200000,45600,44000')

# convert from the string environment variables to other, more convenient, representations
INPUT_SAMPLE_RATES = [int(x) for x in INPUT_SAMPLE_RATES_STR.split(',')]
OUTPUT_SAMPLE_RATES = [int(x) for x in OUTPUT_SAMPLE_RATES_STR.split(',')]

In [59]:
INPUT_TEMPLATE_FILE = "../../hdr/transform/falcon_dsp_predefined_fir_filter_template.h"
OUTPUT_FILE = "../../hdr/transform/falcon_dsp_predefined_fir_filter.h"

input_fd = open(INPUT_TEMPLATE_FILE, 'r')
output_fd = open(OUTPUT_FILE, 'w')

# write the first part of the file
for line in input_fd:
    if "AUTO_GENERATED_COEFFICIENTS_HERE" in line:
        break
    else:
        output_fd.write(line)

In [60]:
for resample_idx in range(len(INPUT_SAMPLE_RATES)):

    INPUT_SAMPLE_RATE = INPUT_SAMPLE_RATES[resample_idx]
    OUTPUT_SAMPLE_RATE = OUTPUT_SAMPLE_RATES[resample_idx]
    
    resampling_ratio = OUTPUT_SAMPLE_RATE / INPUT_SAMPLE_RATE
    ratio = Fraction("%.12f" % (resampling_ratio)).limit_denominator()
    p = ratio.numerator
    q = ratio.denominator
    pqmax = max(p, q)

    # cutoff frequency of the lowpass filter at the high (upsampled) rate
    cutoff_freq = 1 / 2 / pqmax
    filter_order = 2 * 10 * pqmax + 1
    filter_delay = int((filter_order - 1) / p / 2)

    filter_coeffs = float(p) * signal.firls(filter_order, [0, 2.0 * cutoff_freq, 2.0 * cutoff_freq, 1.0], [1.0, 1.0, 0.0, 0.0])
    filter_coeffs = filter_coeffs * signal.kaiser(filter_order, beta=5)

    # print 'header information'
    output_fd.write("        /* INPUT_SAMPLE_RATE:   %12u sps\n" % INPUT_SAMPLE_RATE)
    output_fd.write("         * OUTPUT_SAMPLE_RATE:  %12u sps\n" % OUTPUT_SAMPLE_RATE)
    output_fd.write("         */\n")
    output_fd.write("        {\n");
    output_fd.write("            std::make_pair(%u, %u),\n" % (INPUT_SAMPLE_RATE, OUTPUT_SAMPLE_RATE))
    output_fd.write("            {%u, /* up_rate */\n" % p)
    output_fd.write("             %u, /* down_rate */\n" % q)
    output_fd.write("             std::vector<std::complex<float>>{\n")
    next_line = "                 "
    for coeff_idx in range(len(filter_coeffs)):
        next_line += "{%+01.08f, %01.08f}" % (filter_coeffs[coeff_idx].real, filter_coeffs[coeff_idx].imag)
        if coeff_idx % 4 == 3:
            output_fd.write(next_line + ",\n"); next_line = "                 "
        else:
            next_line += ", "

    output_fd.write(next_line[:-2])
    output_fd.write("             }\n")
    output_fd.write("            } /* end of %u sps -> %u sps */\n" % (INPUT_SAMPLE_RATE, OUTPUT_SAMPLE_RATE))
    
    if resample_idx != (len(INPUT_SAMPLE_RATES) - 1):
        output_fd.write("        },\n");
    else:
        output_fd.write("        }\n");

In [61]:
# write the last part of the file
for line in input_fd:
    output_fd.write(line)
        
input_fd.close()
output_fd.close()